In [1]:
%matplotlib notebook
import firedrake
from firedrake import *
import matplotlib.pyplot as plt
import numpy as np
from constants import *
from hs_solver import *
from phi_solver import *
from model import *
from scipy.io import loadmat

# Thwiaites experiment
The subglacial enviornment of Thwaites Glacier is of much interest to scientists and policy makes as the bed conditions beneath Thwaites control the rate of West Antarctic collapse. Recently, several large sub-glacial lakes have been observed to fill and drain on internanual timescales. 

# Input data for model

In [2]:
x=loadmat('/Volumes/GoogleDrive/My Drive/papers/Thwaites_lakes_p1/thw124.mat')
M=x['M']
dist=M[:,0] # distance along flow line (m)
bed=M[:,1] # bed elevation (m)
surface=M[:,2] # surface elevation (m)
melt_ian=M[:,3] # basal melt rate. (mm/a)
ub_ian=M[:,4] # basal slip speed (m/yr)

# Define model domain

In [3]:
Lx = dist[-1]
nx =len(dist)-1
flux_condition = 0.0
mesh = firedrake.IntervalMesh(nx,Lx)
degree = 1
V_cg = firedrake.FunctionSpace(mesh,"CG",degree)

In [4]:
H=firedrake.Function(V_cg)
Bed=firedrake.Function(V_cg)
Surface=firedrake.Function(V_cg)
width = firedrake.Function(V_cg)
m = firedrake.Function(V_cg)
ub = firedrake.Function(V_cg)
width = firedrake.interpolate(firedrake.Constant(20000.0),V_cg)

In [5]:
Surface.dat.data[:] = surface
Bed.dat.data[:] = bed
ub.dat.data[:]=ub_ian/pcs["spy"]
m.dat.data[:]=melt_ian/pcs["spy"]/1000.0
H = firedrake.interpolate(Surface-Bed,V_cg)

In [6]:
########### Model Initializtion ############

h_init = firedrake.Function(V_cg)
h_init = firedrake.interpolate(Constant(1.0e-10),V_cg)
S_init = firedrake.Function(V_cg)

# Load potential at 0 pressure
phi_m = firedrake.Function(V_cg)
phi_m = firedrake.assemble(Bed*pcs['g']*pcs['rho_water'])
# Ice overburden pressure
p_i = firedrake.Function(V_cg)
p_i = firedrake.assemble(H*pcs['g']*pcs['rho_ice'])
# Enforce 0 pressure bc at margin
bc = firedrake.DirichletBC(V_cg,p_i+phi_m,2)

# Initialize hydropotential
phi_init=firedrake.Function(V_cg)
phi_init=p_i+phi_m

# change constants
pcs["e_v"]=1e-9
pcs["h_r"]=0.05

In [7]:
model_inputs = {}
model_inputs['phi_m'] = phi_m
model_inputs['p_i'] = p_i
model_inputs['phi_0'] = p_i+phi_m
model_inputs['mesh'] = mesh
model_inputs['H'] = H
model_inputs['B'] = Bed
model_inputs['u_b'] = ub
model_inputs['m'] = m
model_inputs['h_init'] = h_init
model_inputs['S_init'] = S_init
model_inputs['phi_init'] = phi_init
model_inputs['d_bcs'] = [bc]
model_inputs['width'] = width
model_inputs['out_dir'] = "outputs/"
model_inputs['constants'] = pcs

In [8]:
# Create the Glads model
thwaites = Glads1DModel(model_inputs)
# End time
T = 5.0*pcs['spy']
# Time step
dt = 60.0*5

In [9]:
############# Run simulation ################

while thwaites.t < T:
    thwaites.step(dt)
    if (int(thwaites.t) % int(30*pcs['spd']) ==0):
        print(thwaites.t)

ConvergenceError: Nonlinear solve failed to converge after 50 nonlinear iterations.
Reason:
   DIVERGED_MAX_IT

In [ ]:
plot(thwaites.S)
plot(thwaites.h)

In [ ]:
plot(thwaites.phi)

In [ ]:
plot(assemble(thwaites.N*-1))

In [ ]:
plot(thwaites.h)